# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [7]:
# import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
from sqlalchemy import create_engine
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import make_scorer, accuracy_score, recall_score, precision_score, fbeta_score, classification_report
import pickle




%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\trekd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\trekd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\trekd\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse.db', engine)
X = df['message']
Y = df.drop(columns=['id', 'message', 'original', 'genre'])

In [9]:
filepath = './data/DisasterResponse.db'
db = filepath.split('/')[-1]

In [10]:
# load data from database
engine = create_engine(f'sqlite:///{filepath}')
df = pd.read_sql_table(db, engine)
X = df['message']
Y = df.drop(columns=['id', 'message', 'original', 'genre'])

### 2. Write a tokenization function to process your text data

In [11]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english')) 
pattern = re.compile(r'[^a-zA-Z0-9]')

def tokenize(text):    
    text = pattern.sub(' ', text)
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(w, pos='v') for w in tokens if w not in stop_words]    
   # tokens = [stemmer.stem(w) for w in tokens]
    
    return tokens

In [12]:
# ind = 10
# print(tokenize(X.to_list()[ind]))
# print(X[ind])

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, token_pattern=None, lowercase=True)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(ComplementNB())),
])

param = {
    'vect__ngram_range': [(1,1), (1,2), (2,2)],
    'vect__max_df':[0.6, 0.8, 1.0],
    'tfidf__norm': ['l1', 'l2', None],
    'tfidf__use_idf': [True, False],
    'clf__estimator__alpha': (0.2, 0.5, 0.8, 2.0), 
}

multi_accuracy = make_scorer(accuracy_score)
multi_precision = make_scorer(precision_score, average='macro', zero_division=0)
multi_recall = make_scorer(recall_score, average='macro', zero_division=0)
multi_fbeta = make_scorer(fbeta_score, beta=2, average='macro', zero_division=0)

# scoring = {'acc': multi_accuracy,
#           # 'prec': multi_precision,
#            'rec': multi_recall,
#            #'f1': 'f1',
#            'fbeta_2': multi_fbeta,
#             #'avg_prec':avg_precision,
#           }

In [14]:
cv = RandomizedSearchCV(pipeline,
                      param,                      
                      verbose=5,
                      error_score='raise',
                      return_train_score=False,
                      n_jobs=-1,
                      n_iter=25,
                      scoring=multi_fbeta,
                      random_state=41,
                      #refit='fbeta_2',
                     )

In [15]:
# cv = GridSearchCV(pipeline,
#                   param,                  
#                   verbose=5,
#                   error_score='raise',
#                   return_train_score=False,
#                   n_jobs=-1,
#                   scoring=scoring,
#                   refit='fbeta_2',
#                  # random_state=41
#                  )

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, random_state=41)

In [17]:
start = time.time()
cv.fit(X_train, y_train)
end = time.time()
total = (end-start)/60
print(f'Complete in {total} minutes.')

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Complete in 1.9950464685757956 minutes.


In [18]:
pd.DataFrame(cv.cv_results_).sort_values('rank_test_score').head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_vect__ngram_range,param_vect__max_df,param_tfidf__use_idf,param_tfidf__norm,param_clf__estimator__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
23,3.642257,0.054852,0.877253,0.026840,"(1, 1)",0.6,True,None,0.8,"{'vect__ngram_range': (1, 1), 'vect__max_df': ...",0.381971,0.382329,0.383328,0.383107,0.380789,0.382305,0.000905,1
21,3.610542,0.049230,0.868077,0.023875,"(1, 1)",0.8,True,None,2.0,"{'vect__ngram_range': (1, 1), 'vect__max_df': ...",0.381276,0.379366,0.379261,0.386103,0.375706,0.380342,0.003396,2
19,4.683272,0.110331,1.042810,0.026870,"(1, 2)",0.8,False,None,0.2,"{'vect__ngram_range': (1, 2), 'vect__max_df': ...",0.350067,0.344692,0.347797,0.365199,0.353617,0.352274,0.007089,3
16,4.769450,0.035380,1.050181,0.034015,"(1, 2)",0.6,False,None,0.2,"{'vect__ngram_range': (1, 2), 'vect__max_df': ...",0.350067,0.344692,0.347797,0.365199,0.353617,0.352274,0.007089,3
4,4.869972,0.089224,1.052983,0.028198,"(1, 2)",1.0,True,None,2.0,"{'vect__ngram_range': (1, 2), 'vect__max_df': ...",0.341148,0.338574,0.345176,0.359895,0.350157,0.346990,0.007547,5


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [19]:
pred = pd.DataFrame(cv.predict(X_test))

target_names = y_test.columns

for i, col in enumerate(y_test):
    print(f'-------{target_names[i]}-{i}-------\n')
    print(classification_report(y_test[col], pred[i]))

-------related-0-------

              precision    recall  f1-score   support

           0       0.53      0.66      0.59      1553
           1       0.89      0.82      0.85      5001

    accuracy                           0.78      6554
   macro avg       0.71      0.74      0.72      6554
weighted avg       0.80      0.78      0.79      6554

-------request-1-------

              precision    recall  f1-score   support

           0       0.95      0.78      0.86      5407
           1       0.43      0.80      0.56      1147

    accuracy                           0.78      6554
   macro avg       0.69      0.79      0.71      6554
weighted avg       0.86      0.78      0.80      6554

-------offer-2-------

              precision    recall  f1-score   support

           0       1.00      0.98      0.99      6527
           1       0.04      0.26      0.07        27

    accuracy                           0.97      6554
   macro avg       0.52      0.62      0.53      6554
w

### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
# parameters = 

# cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [21]:
accuracy = multi_accuracy(cv, X_test, y_test)
precision = multi_precision(cv, X_test, y_test)
recall = multi_recall(cv, X_test, y_test)

print(f'macro-averaged accuracy: {accuracy:.2f} -- macro-averaged precision: {precision:.2f} -- macro-averaged recall: {recall:.2f}')

macro-averaged accuracy: 0.07 -- macro-averaged precision: 0.22 -- macro-averaged recall: 0.53


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [22]:
filename = r'./models/model.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.